In [30]:
!pip install tqdm
!pip install pytorch-pretrained-bert
!pip install keras
!pip install tensorflow

You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [31]:
import logging
import os
import argparse
import random
from tqdm import tqdm, trange
import csv
import pandas as pd
import re

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
train = pd.read_csv("InferSent/encoder/data/train.tsv",sep='\t')
train.head(3)

,ID,Question Stem,Choices,Complete Question,Answer Key
0,7-980,The sun is responsible for,(A) puppies learning new tricks (B) children g...,The sun is responsible for (A) puppies learnin...,D
1,7-584,When standing miles away from Mount Rushmore,(A) the mountains seem very close (B) the moun...,When standing miles away from Mount Rushmore (...,D
2,7-870,When food is reduced in the stomach,(A) the mind needs time to digest (B) take a s...,When food is reduced in the stomach (A) the mi...,C


In [32]:
#Create new features from the answer choices  
for index, row in train.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    train.set_value(index,'OptionA',ans[1])
    train.set_value(index,'OptionB',ans[3])
    train.set_value(index,'OptionC',ans[5])
    train.set_value(index,'OptionD',ans[7])
    
#Find the answer key and replace it with the value   
for index, row in train.iterrows():
    ans = re.split("(\([A-Z]\))", row['Choices'])
    ans.remove('')
    if row['Answer Key'] == 'A':
        train.set_value(index,'Result',ans[1])
    elif row['Answer Key'] == 'B':
        train.set_value(index,'Result',ans[3])
    elif row['Answer Key'] == 'C':
        train.set_value(index,'Result',ans[5])
    elif row['Answer Key'] == 'D':
        train.set_value(index,'Result',ans[7]) 

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: FutureWarning: set_value

In [33]:
num_of_categories = 4957
shuffled = train.reindex(np.random.permutation(train.index))
a = shuffled[shuffled['Answer Key'] == 'A'][:num_of_categories]
b = shuffled[shuffled['Answer Key'] == 'B'][:num_of_categories]
c = shuffled[shuffled['Answer Key'] == 'C'][:num_of_categories]
d = shuffled[shuffled['Answer Key'] == 'D'][:num_of_categories]
concated = pd.concat([a,b,c,d], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [34]:
from keras.utils.np_utils import to_categorical
#One-hot encode the labels
concated.loc[concated['Answer Key'] == 'A', 'LABEL'] = 0
concated.loc[concated['Answer Key'] == 'B', 'LABEL'] = 1
concated.loc[concated['Answer Key'] == 'C','LABEL'] = 2
concated.loc[concated['Answer Key'] == 'D','LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)    

3355    2
2405    1
4373    3
107     0
3802    3
39      0
2522    1
3873    3
1762    1
2224    1
Name: LABEL, dtype: int64
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [35]:
concated.head(10)

,ID,Question Stem,Choices,Complete Question,Answer Key,OptionA,OptionB,OptionC,OptionD,Result,LABEL
3355,12-1201,Which are likely slightly larger in size than ...,(A) solid steel (B) solid wood (C) solid glaci...,Which are likely slightly larger in size than ...,C,solid steel,solid wood,solid glaciers,solid mercury,solid glaciers,2
2405,13-508,What does a stove generate for cooking?,(A) ice (B) scorching temperatures (C) cold (D...,What does a stove generate for cooking? (A) ic...,B,ice,scorching temperatures,cold,freezing,scorching temperatures,1
4373,9-1064,a plant needs this to survive,(A) bugs to eat (B) liquid food (C) finger foo...,a plant needs this to survive (A) bugs to eat ...,D,bugs to eat,liquid food,finger food,nutritional material,nutritional material,3
107,7-590,An example of weathering is,(A) sand (B) sharp boulders (C) tall trees (D)...,An example of weathering is (A) sand (B) sharp...,A,sand,sharp boulders,tall trees,coral reefs,sand,0
3802,12-918,Echolocation is when animals detect objects by...,(A) smell (B) vision (C) sight (D) screeching,Echolocation is when animals detect objects by...,D,smell,vision,sight,screeching,screeching,3
39,10-397,A large turbine in a field can produce electri...,(A) a breeze is floating (B) a child is crying...,A large turbine in a field can produce electri...,A,a breeze is floating,a child is crying,a dog is barking,clouds are very thin,a breeze is floating,0
2522,8-339,Peat is an important factor when,(A) Driving a new car (B) Putting a fresh rasp...,Peat is an important factor when (A) Driving a...,B,Driving a new car,Putting a fresh raspberry seed in soil,Taking the temperature for the day,Having a large breakfast,Putting a fresh raspberry seed in soil,1
3873,9-976,Leaves are the part of a plant with the most a...,(A) root (B) bud (C) bacteria (D) specialized ...,Leaves are the part of a plant with the most a...,D,root,bud,bacteria,specialized compartment,specialized compartment,3
1762,10-1041,Precipitation is the amount of,(A) dew point (B) raindrops (C) flooding (D) b...,Precipitation is the amount of (A) dew point (...,B,dew point,raindrops,flooding,barometric pressure,raindrops,1
2224,10-525,Where could you expect to see a quail sleeping?,(A) box (B) field (C) ocean (D) tree,Where could you expect to see a quail sleeping...,B,box,field,ocean,tree,field,1


In [36]:
sentences = []
with open('InferSent/encoder/data/openbook.txt') as f:
    for line in f:
        sentences.append(line.strip())
print(len(sentences))

1326


In [37]:
class QAExample(object):
    """A single training/test example for the SWAG dataset."""
    def __init__(self,
                 QA_id,
                 context_sentence,
                 question_stem,
                 ending_0,
                 ending_1,
                 ending_2,
                 ending_3,
                 label = None):
        self.qa_id = QA_id
        self.context_sentence = context_sentence
        self.question_stem = question_stem
        self.endings = [
            ending_0,
            ending_1,
            ending_2,
            ending_3,
        ]
        self.label = label

class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label        

In [38]:

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x            

In [39]:
 examples = [
        QAExample(
            QA_id = concated['ID'].values,
            context_sentence = sentences,
            question_stem = concated["Question Stem"].values,
            ending_0 = concated['OptionA'].values,
            ending_1 = concated['OptionB'].values,
            ending_2 = concated['OptionC'].values,
            ending_3 = concated['OptionD'].values,
            label = concated["LABEL"].values
        )
    ]

In [40]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


01/27/2019 19:47:05 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ec2-user/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [19]:
from keras.preprocessing.sequence import pad_sequences
features = []
max_seq_length = 128
for example_index, example in enumerate(examples):
    context_sentence_tokens = tokenizer.tokenize(''.join(example.context_sentence))
    question_stem_tokens = tokenizer.tokenize(''.join(example.question_stem))
   
    
    
    choices_features = [] 
    for ending_index, ending in enumerate(example.endings):
        context_tokens_sentence = context_sentence_tokens[:]
        ending_tokens = question_stem_tokens + tokenizer.tokenize(''.join(ending))        
        _truncate_seq_pair(context_tokens_sentence, ending_tokens, max_seq_length - 3)
        
        
        tokens = ["[CLS]"] + context_tokens_sentence + ["[SEP]"] + ending_tokens + ["[SEP]"]
                                                          
                                                                  
        segment_ids = [0] * (len(context_tokens_sentence) + 2) + [1] * (len(ending_tokens) + 1)   
     
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
      
        input_mask = [1] * len(input_ids)
      

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        choices_features.append((tokens, input_ids, input_mask, segment_ids))
        
                                                                  
                                                                  
        labels = example.label
         
                            
        features.append(
            InputFeatures(
                example_id = example.qa_id,
                choices_features = choices_features,
                label = labels
            )
        )  
         

In [41]:
import os

local_rank = -1
fp16 = ""
no_cuda = ""
gradient_accumulation_steps = 1
seed = 42
train_batch_size = 32
do_train=True
do_eval=True
num_train_epochs = 30
bert_model = 'bert-base-uncased'

output_dir=os.getcwd() + "models"

if local_rank == -1 or no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda",local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
    device, n_gpu, bool(local_rank != -1), fp16))

if gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        gradient_accumulation_steps))

train_batch_size = int(train_batch_size / gradient_accumulation_steps)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

if not do_train and not do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")

if os.path.exists(output_dir) and os.listdir(output_dir):
    raise ValueError("Output directory ({}) already exists and is not empty.".format(output_dir))
os.makedirs(output_dir, exist_ok=True)

num_train_steps = int(
        len(train) / train_batch_size / gradient_accumulation_steps * num_train_epochs)


# Prepare model
model = BertForMultipleChoice.from_pretrained('bert-base-uncased',
                                              cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(local_rank),
                                              num_choices=4)

# if fp16:
#     model.half()
# model.to(device)
# if local_rank != -1:
#     try:
#         from apex.parallel import DistributedDataParallel as DDP
#     except ImportError:
#         raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

#     model = DDP(model)
# elif n_gpu > 1:
#     model = torch.nn.DataParallel(model)

01/27/2019 19:47:22 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: 
01/27/2019 19:47:22 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/ec2-user/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/27/2019 19:47:22 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/ec2-user/.pytorch_pretrained_bert/distributed_-1/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpvxww5oxk
01/27/2019 19:47:26 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0

In [42]:
# import os

# local_rank = -1
# fp16 = ""
# no_cuda = ""
# gradient_accumulation_steps = 1
# seed = 42
# train_batch_size = 32
# do_train=True
# do_eval=True
# num_train_epochs = 30
# bert_model = 'bert-base-uncased'

# output_dir=os.getcwd() + "models"



# model = BertForMultipleChoice.from_pretrained('bert-base-uncased',
#                                               cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(local_rank),
#                                               num_choices=4)


In [69]:
learning_rate = 5e-5
warmup_proportion = 0.1
loss_scale = 0
train_features = features


# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps

if local_rank != -1:
    t_total = t_total // torch.distributed.get_world_size()
if fp16:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=learning_rate,
                          bias_correction=False,
                          max_grad_norm=1.0)
    if loss_scale == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=loss_scale)
else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=learning_rate,
                     warmup=warmup_proportion,
                     t_total=t_total)  

input_ids = select_field(train_features, 'input_ids')
flat_list_input_ids = []
for item in input_ids:
    flat_list_input_ids.append(item)

input_mask = select_field(train_features, 'input_mask')
flat_list_input_mask = []
for item in input_mask:
    flat_list_input_mask.append(item)


segment_ids = select_field(train_features, 'segment_ids')
flat_list_segment_ids = []
for item in segment_ids:
    flat_list_segment_ids.append(item)

# labels_y=[]
# for item in train_features:
#     for label in item.label:
#         labels_y.append(label)

global_step = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)


all_input_ids = torch.FloatTensor(input_ids[3])
all_input_mask = torch.tensor(input_mask[3])
all_segment_ids = torch.tensor(segment_ids[3])
all_label = torch.tensor([f.label for f in train_features])
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)

all_input_ids.size(),all_input_mask.size(),all_segment_ids.size(),all_label.size()


01/27/2019 20:08:18 - INFO - __main__ -   ***** Running training *****
01/27/2019 20:08:18 - INFO - __main__ -     Num examples = 4957
01/27/2019 20:08:18 - INFO - __main__ -     Batch size = 32
01/27/2019 20:08:18 - INFO - __main__ -     Num steps = 4647


(torch.Size([4, 128]),
 torch.Size([4, 128]),
 torch.Size([4, 128]),
 torch.Size([4, 4957]))

In [22]:
# from torch.optim import Adam

# learning_rate = 5e-5
# warmup_proportion = 0.1
# loss_scale = 0
# train_features = features

# # Prepare optimizer
# FULL_FINETUNING = True
# if FULL_FINETUNING:
#     param_optimizer = list(model.named_parameters())
#     no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#     optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]
#     t_total = num_train_steps
# else:
#     param_optimizer = list(model.classifier.named_parameters()) 
#     optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
# optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)



# input_ids = select_field(train_features, 'input_ids')
# flat_list_input_ids = []
# for item in input_ids:
#     flat_list_input_ids.append(item)

# input_mask = select_field(train_features, 'input_mask')
# flat_list_input_mask = []
# for item in input_mask:
#     flat_list_input_mask.append(item)


# segment_ids = select_field(train_features, 'segment_ids')
# flat_list_segment_ids = []
# for item in segment_ids:
#     flat_list_segment_ids.append(item)

# # labels_y=[]
# # for item in train_features:
# #     for label in item.label:
# #         labels_y.append(label)

# global_step = 0
# logger.info("***** Running training *****")
# logger.info("  Num examples = %d", len(train))
# logger.info("  Batch size = %d", train_batch_size)
# logger.info("  Num steps = %d", num_train_steps)


# all_input_ids = torch.tensor(flat_list_input_ids[3], dtype=torch.long)
# all_input_mask = torch.tensor(flat_list_input_mask[3], dtype=torch.long)
# all_segment_ids = torch.tensor(flat_list_segment_ids[3], dtype=torch.long)
# all_label = torch.tensor([f.label for f in train_features], dtype=torch.long)
# train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label)


In [70]:

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)
  
for _ in trange(3, desc="Epoch"):
    model.train() 
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        
        loss = model(input_ids, segment_ids, input_mask, label_ids)

        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.

        if fp16 and loss_scale != 1.0:
            # rescale loss for fp16 training
            # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
            loss = loss * loss_scale

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if fp16:
            optimizer.backward(loss)
        else:
            loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            # modify learning rate with special warm up BERT uses
            lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/THC/generic/THCTensorCopy.cpp:20

In [219]:
all_segment_ids.shape, all_input_ids.shape

(torch.Size([10, 128]), torch.Size([10, 128]))

In [229]:
input_ids = select_field(train_features, 'input_ids')
len(input_ids[1])

2